In [ ]:
from astropy.io import fits
from astropy.coordinates import Angle
import matplotlib.pyplot as pl
import numpy as np
fromgala.coordinates import vhel_to_vgsr
importgala.potential as sp
%matplotlib inline

In [ ]:
# hdulist = fits.open("/Users/adrian/Data/APOGEE_DR10/allStar-v304.fits")
hdulist = fits.open("/Users/adrian/Data/APOGEE_DR12/allStar-v603.fits")
data = hdulist[1].data

In [ ]:
sorted(data.dtype.names)

In [ ]:
ix = (data['VSCATTER'] < 1.) & (data['VERR'] < 1.)
rv_data = data[ix]

In [ ]:
pl.plot(rv_data['GLON'], rv_data['GLAT'], ls='none')
pl.xlim(20,34)
pl.ylim(-10,10)

In [ ]:
near_plane = rv_data[np.abs(rv_data['GLAT'] < 1.)]

In [ ]:
pl.plot(near_plane['GLON'], near_plane['VHELIO_AVG'], ls='none')
pl.xlim(20,34)
pl.ylim(50,250)

In [ ]:
ix = (data['ALPHAFE'] > -1000) & (data['ALPHAFE'] < 10) & (data['ALPHAFE'] != 0) & (data['METALS'] > -1000)
alpha_fe = data['ALPHAFE'][ix]
fe_h = data['METALS'][ix]

plt.figure(figsize=(8,8))
plt.plot(fe_h, alpha_fe, linestyle='none', alpha=0.2)

In [ ]:
badflag = sum(2**np.array([3,7,19,23]))
badflag

In [ ]:
# qmask = (data['PM_SRC'] != 'none') & \
#         (data['VERR'] < 5.) & \
#        ((data['ASPCAPFLAG'] & badflag) == 0)# & \
qmask = (data['VERR'] < 5.) & \
        ((data['ASPCAPFLAG'] & badflag) == 0) &\
        np.isfinite(data['J']) & np.isfinite(data['H']) & np.isfinite(data['K'])
good_data = data[qmask]
        
print sum(~qmask), "excluded"
print sum(qmask), "included"

In [ ]:
plt.plot(good_data['GLON'], good_data['GLAT'], 
         marker='.', linestyle='none', alpha=0.1)

In [ ]:
from astropy.coordinates import Angle

l = Angle(good_data['GLON']*u.deg)
b = Angle(good_data['GLAT']*u.deg)

fig = plt.figure()
ax = fig.add_subplot(111,projection='hammer')
ax.plot(l.wrap_at(180*u.deg).radian, b.radian,
        marker='.', linestyle='none')

In [ ]:
vgsr = vhel_to_vgsr(Angle(good_data['GLON']*u.deg), 
                    Angle(good_data['GLAT']*u.deg), 
                    good_data['VHELIO_AVG']*u.km/u.s)

In [ ]:
fig,axes = plt.subplots(3,1,figsize=(8,12))

axes[0].plot(good_data['J'], vgsr.value, 
             marker='.', linestyle='none', alpha=0.1)
axes[1].plot(good_data['H'], vgsr.value, 
             marker='.', linestyle='none', alpha=0.1)
axes[2].plot(good_data['K'], vgsr.value, 
             marker='.', linestyle='none', alpha=0.1)

for ax in axes:
    ax.set_ylim(-300,300)

M giant selection from http://arxiv.org/pdf/1310.7565v2.pdf:

(J − K) > 1.02  
(J − H) < 0.561 × (J − K)o + 0.46  
(J − H) > 0.561 × (J − K)o + 0.185

In [ ]:
Mgiant_mask = ((good_data['J']-good_data['K']) > 1.02) & \
              ((good_data['J']-good_data['H']) < (0.561*(good_data['J']-good_data['K'])+0.46)) & \
              ((good_data['J']-good_data['H']) > (0.561*(good_data['J']-good_data['K'])+0.185))
print sum(Mgiant_mask)

In [ ]:
Mgiants = good_data[Mgiant_mask]
vgsr = vhel_to_vgsr(Angle(Mgiants['GLON']*u.deg), 
                    Angle(Mgiants['GLAT']*u.deg), 
                    Mgiants['VHELIO_AVG']*u.km/u.s)

In [ ]:
fig,axes = plt.subplots(3,1,figsize=(8,12))

axes[0].plot(Mgiants['J'], vgsr.value, 
             marker='.', linestyle='none', alpha=0.2)
axes[1].plot(Mgiants['H'], vgsr.value, 
             marker='.', linestyle='none', alpha=0.2)
axes[2].plot(Mgiants['K'], vgsr.value, 
             marker='.', linestyle='none', alpha=0.2)

for ax in axes:
    ax.set_ylim(-300,300)

In [ ]:
M_K = 3.26 - 9.42*(good_data['J']-good_data['K']) # Sharma 2010
mu = good_data['K'] - M_K
dist = 10**(1 + mu/5)*u.pc
dist = dist.to(u.kpc)

dist_mask = dist < 100*u.kpc

In [ ]:
plt.hist(M_K, bins=np.linspace(-30,3,25));

In [ ]:
dist_data = good_data[dist_mask]
good_dist = dist[dist_mask]

In [ ]:
plt.hist(good_dist, bins=100);

In [ ]:
theta = 122.932*u.deg
dec_ngp = 27.12825*u.deg
ra_ngp = 192.85948*u.deg

ra = dist_data['RA']*u.deg
dec = dist_data['DEC']*u.deg
b = dist_data['GLAT']*u.deg
pmra = dist_data['PMRA']
pmdec = dist_data['PMDEC']
        
cosphi = (np.sin(dec_ngp) - np.sin(dec)*np.sin(b))/np.cos(dec)/np.cos(b)
sinphi = np.sin(ra-ra_ngp)*np.cos(dec_ngp)/np.cos(b)

all_R = np.array([[cosphi,sinphi],[-sinphi,cosphi]])
pml,pmb = np.array([np.dot(R,pm) for pm,R in zip(np.array([pmra,pmdec]).T,all_R.T)]).T

In [ ]:
x,y,z,vx,vy,vz = hel_to_gc(dist_data['GLON']*u.deg, dist_data['GLAT']*u.deg, good_dist, 
                           pml*u.mas/u.yr, pmb*u.mas/u.yr, dist_data['VHELIO_AVG']*u.km/u.s)

zmask = ~np.isnan(vx.value)
x = x[zmask]
y = y[zmask]
z = z[zmask]
vx = vx[zmask]
vy = vy[zmask]
vz = vz[zmask]

In [ ]:
plt.plot(x.value, y.value,
         marker='.', alpha=0.1, linestyle='none')
plt.xlim(-75,75)
plt.ylim(-20,30)

In [ ]:
plt.plot(vx.value, vz.value,
         marker='.', alpha=0.1, linestyle='none')
plt.xlim(-400,400)
plt.ylim(-400,400)

---

Some interesting clumps in velocity space

In [ ]:
plt.plot(vx.value, vz.value,
         marker='.', alpha=0.5, linestyle='none')
plt.xlim(-13,-9)
plt.ylim(-145,-125)

In [ ]:
idx = (vx.value > -13) & (vx.value < -9) & \
      (vy.value > 204.) & (vy.value < 208.) & \
      (vz.value > -145) & (vz.value < -125) 

plt.figure()
plt.plot(x.value, z.value,
         marker='.', alpha=0.1, linestyle='none')
plt.plot(x.value[idx], z.value[idx],
        marker='.', alpha=1., linestyle='none')
plt.xlim(-9,0)
plt.ylim(-5,15)

plt.figure()
plt.plot(x.value, y.value,
         marker='.', alpha=0.1, linestyle='none')
plt.plot(x.value[idx], y.value[idx],
        marker='.', alpha=1., linestyle='none')
plt.xlim(-9,0)
plt.ylim(-3,3)

plt.figure()
plt.plot(y.value, z.value,
         marker='.', alpha=0.1, linestyle='none')
plt.plot(y.value[idx], z.value[idx],
        marker='.', alpha=1., linestyle='none')
plt.xlim(-3,3)
plt.ylim(-5,15)

In [ ]:
usys = [u.Myr,u.kpc,u.radian,u.M_sun]
potential = sp.CompositePotential(units=usys)
potential["disk"] = sp.MiyamotoNagaiPotential(units=usys,
                                   m=1.E11*u.M_sun,
                                   a=6.5*u.kpc,
                                   b=0.26*u.kpc,
                                   r_0=[0.,0.,0.]*u.kpc)

potential["bulge"] = sp.HernquistPotential(units=usys,
                               m=1.E11*u.M_sun,
                               c=0.7*u.kpc)

potential["halo"] = sp.SphericalNFWPotential(units=usys, 
                                             log_m=np.log(2.5E11),
                                             a=6*u.kpc)

In [ ]:
r = np.vstack((x.value,y.value,z.value)).T.copy()*u.kpc

In [ ]:
energy_data = good_data[zmask]
pvalue = potential.value_at(r)
Energy = pvalue + 0.5*(vx**2+vy**2+vz**2)

In [ ]:
plt.hist(energy_data['metals'], bins=np.linspace(-5, 1, 100));

In [ ]:
_idx = (energy_data['metals'] < 1) & (Energy.value < 50)

plt.semilogx(Energy.value[_idx]+1, energy_data['metals'][_idx], 
             marker='.', alpha=0.1, linestyle='none')
plt.axvline(-0.185+1.)
plt.xlim(0.7, 3)
plt.xlabel("Energy")
plt.ylabel("[Fe/H]")

In [ ]:
r = np.vstack((x.value,y.value,z.value)).T.copy()*x.unit
v = np.vstack((vx.value,vy.value,vz.value)).T.copy()*vx.unit
L = np.sqrt(np.sum(np.array([np.cross(rr,vv) for rr,vv in zip(r,v)])**2, axis=-1))

plt.figure(figsize=(8,8))
plt.loglog(Energy.value+1, L, 
           marker='.', alpha=0.1, linestyle='none')
plt.loglog(Energy.value[idx]+1, L[idx], 
           marker='.', alpha=0.1, linestyle='none')
#axvline(-0.185+1.)
plt.xlim(0.1, 10)
plt.ylim(1E2, 1E6)
plt.xlabel("Energy")
plt.ylabel("Angular mom.")

In [ ]:
r = np.vstack((x.value,y.value,z.value)).T.copy()*x.unit
v = np.vstack((vx.value,vy.value,vz.value)).T.copy()*vx.unit
L = np.sqrt(np.sum(np.array([np.cross(rr,vv) for rr,vv in zip(r,v)])**2, axis=-1))

plt.figure(figsize=(8,8))
plt.loglog(Energy.value+1, L, 
           marker='.', alpha=0.1, linestyle='none')
plt.loglog(Energy.value[idx]+1, L[idx], 
           marker='.', alpha=0.1, linestyle='none')
#axvline(-0.185+1.)
plt.xlim(0.7, 2)
plt.ylim(1E2, 1E5)
plt.xlabel("Energy")
plt.ylabel("Angular mom.")

In [ ]:
#idx = ((Energy.value+1) < 0.83) & (L < 2500) & (L > 1000)
idx = ((Energy.value+1) < 1.1) 

figure(figsize=(3,3))
loglog(Energy.value[idx]+1, L[idx], 
         marker='.', alpha=0.1, linestyle='none')
#axvline(-0.185+1.)
xlim(0.7, 2)
ylim(1E2, 1E5)
xlabel("Energy")
ylabel("Angular mom.")

figure(figsize=(8,8))
plot(x[idx].value, z[idx].value,
     marker='.', alpha=0.1, linestyle='none')


In [ ]:
idx = (np.sqrt(x**2+y**2+z**2) > 20.*u.kpc)

r = np.vstack((x.value,y.value,z.value)).T.copy()*x.unit
v = np.vstack((vx.value,vy.value,vz.value)).T.copy()*vx.unit
L = np.sqrt(np.sum(np.array([np.cross(rr,vv) for rr,vv in zip(r,v)])**2, axis=-1))

figure(figsize=(8,8))
loglog(Energy.value[idx]+1, L[idx], 
         marker='.', alpha=0.1, linestyle='none')
#axvline(-0.185+1.)
xlim(0.9, 1.1)
ylim(1E3, 3E4)
xlabel("Energy")
ylabel("Angular mom.")

In [ ]:
# idx = (Energy.value+1) > 0.98 #(np.sqrt(x**2+y**2+z**2) > 20.*u.kpc) & (L < 2700)
idx = (L > (200000*(Energy.value+1)-190000)) & ((Energy.value+1) > 0.98)

r = np.vstack((x.value,y.value,z.value)).T.copy()*x.unit
v = np.vstack((vx.value,vy.value,vz.value)).T.copy()*vx.unit
L = np.sqrt(np.sum(np.array([np.cross(rr,vv) for rr,vv in zip(r,v)])**2, axis=-1))

figure(figsize=(6,6))
loglog(Energy.value[idx]+1, L[idx], 
         marker='.', alpha=0.1, linestyle='none')
xlim(0.9, 1.1)
ylim(1E3, 3E4)
xlabel("Energy")
ylabel("Angular mom.")

fig,axes = subplots(2,2,figsize=(12,12),sharex='col',sharey='row')
axes[0,0].plot(x[idx], y[idx],
               marker='.', alpha=0.2, linestyle='none')
axes[0,1].set_visible(False)
axes[1,0].plot(x[idx], z[idx],
               marker='.', alpha=0.2, linestyle='none')
axes[1,1].plot(y[idx], z[idx],
               marker='.', alpha=0.2, linestyle='none')
axes[0,0].set_xlim(-50,50)

# idx2 = (L < (200000*(Energy.value+1)-190000)) & ((Energy.value+1) > 0.98)
# axes[0,0].plot(x[idx2], y[idx2],
#                marker='.', alpha=0.2, linestyle='none')
# axes[0,1].set_visible(False)
# axes[1,0].plot(x[idx2], z[idx2],
#                marker='.', alpha=0.2, linestyle='none')
# axes[1,1].plot(y[idx2], z[idx2],
#                marker='.', alpha=0.2, linestyle='none')

In [ ]:
plot